In [ ]:
## De functie die tf-records leest en ze uiteindelijk weer wegschrijft (maar dan met minder labels) heet readTfRecord
## Deze functie roept andere functies aan: getLabels en getLabelIndices

import tensorflow as tf
import numpy as np
import pandas as pd
import re
from tensorflow import gfile
#from collections import Counter
#import readers
#import utils

In [ ]:
train_data_pattern = 'tfrecords/train_data_old/*.tfrecord'
train_data_folder_new = 'train_data_new'

In [ ]:
## Open het oude csv-bestand en het aangepaste csv-bestand waarin de superklassen staan
mid_to_label_old = pd.read_csv('csv_files/class_labels_indices.csv')
mid_to_label_new = pd.read_csv('csv_files/class_labels_indices_new.csv')

In [ ]:
#### Zoek bij een oude label (van de 527) het nieuwe label

def getNewLabel(label_old):
    ## Get m-id that matches with index
    mid = mid_to_label_old.loc[mid_to_label_old['index']==label_old, 'mid']

    ## Find the new label belonging to this m-id
    label_new = mid_to_label_new.loc[mid_to_label_old['mid']==str(mid.values[0]), 'index']
                    
    return label_new

In [ ]:
#### Lees tf-records en iterate over de examples#### Lee 
#### Voor elk example wordt de video-id gepakt en de bijbehorende lijst m-id's gezocht (in balanced_segment.csv)

## One tf-record has several examples
## This function checks for each example if it has a label we are interested in
## If so, we compute the one-hot vector
## Input: filename of tf-record
## Result: write new tf-record

def readWriteTfRecord(tfrecords_filename):
    
    ## Create writer that writes new tf-record in train_data_folder_new
    path_tfrecord_new = str(tfrecords_filename.split('\\')[0] + '/' + train_data_folder_new + '/' + tfrecords_filename.split('\\')[2])
    writer = tf.python_io.TFRecordWriter(path_tfrecord_new)

    print(path_tfrecord_new)
    ## Iterator that goes through all examples of tf-record
    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

    for string_record in record_iterator:
        
        ## Take one example
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
 
        ## Get labels and find for each label the m-code in class_label_indices_old
        labels = example.context.feature['labels'].int64_list.value
        
        new_labels = []
        for label in labels:
            ## hij returnt een set van cellen (of leeg of van 1 rij)
            label_new = getNewLabel(label)
            if(label_new.shape[0]>0):
                new_labels.append(label_new.values[0])

        ## If any of the labels left (for our model), write new tf-record
        if len(new_labels) > 0:

            print("New labels: " + str(new_labels))
        
            audio_embedding = example.feature_lists.feature_list['audio_embedding']
            feature_lists = tf.train.FeatureLists(feature_list={"audio": audio_embedding})
            
            label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=labels))
            context_feats = tf.train.Features(feature={"labels": label_feat})
            
            seqExample = tf.train.SequenceExample(context=context_feats, feature_lists=feature_lists)
            
#             try:
#                 writer
#             except NameError:
#                 writer = tf.python_io.TFRecordWriter(train_data_folder_new)
            
            ## Write the example
            writer.write(seqExample.SerializeToString())
            print('Example gemaakt! Video-id: ' + str(example.context.feature['video_id'].bytes_list.value))

    writer.close()
    
# try:
#     if (writer):

# except NameError:
#         print()

In [ ]:
train_data_pattern = 'tfrecords/train_data_old/*.tfrecord'

######################## This code will finally invoke all the other functions

## Read all train files
files = gfile.Glob(train_data_pattern)

for file in files:
    
    ## Read a file and rewrite it
    readWriteTfRecord(file)

In [ ]:
# Helper: show Examples in tfrecord file
filename = 'tfrecords/train_data_new/_1.tfrecord'
record_iterator = tf.python_io.tf_record_iterator(path=filename)

for string_record in record_iterator:

    ## Take one example
    example = tf.train.SequenceExample()
    example.ParseFromString(string_record)
    print(example)